In [3]:
import ipywidgets as widgets
from IPython.display import display
import datetime, json, uuid

# ============================================================
# PRO114 – Elaboración de Informes de Falla según Norma Técnica
# Formato estándar (UI estilo PRO115/PRO173) - SIN botón STOP
# ============================================================

EN_CURSO = "EN_CURSO"
BLOQUEADO = "BLOQUEADO"
FINALIZADO = "FINALIZADO"

def _now_iso():
    return datetime.datetime.now().isoformat(timespec="seconds")

# ------------------------------------------------------------
# ZONA EDITABLE (cambia textos / checklist / validaciones aquí)
# ------------------------------------------------------------

def _es_opcional_por_si_aplica(texto: str) -> bool:
    """Si el texto contiene 'si aplica' en cualquier posición, no es obligatorio."""
    return "si aplica" in (texto or "").lower()

# Inputs opcionales (según pedido): incluir los primeros 3, excluir los 2 últimos sugeridos antes
INPUTS_OPCIONALES = {
    "hora_evento": {"label": "Hora/fecha del evento (opcional)", "required": False},
    "numero_aviso_sigo": {"label": "Número Aviso SIGO (opcional)", "required": False},
    "numero_neomante": {"label": "Número correlativo Neomante (opcional)", "required": False},
}

# NODOS lineales (2.1.1 a 2.1.11). Se excluyen 2.1.12 y 2.1.13.
NODOS = {
    "T1": {
        "type": "task",
        "titulo": "1 Informar desconexión",
        "rol": "Operador o Jefe de Turno de SCC u Operador del COC",
        "descripcion": (
            "Inmediatamente ocurrida la desconexión intempestiva de la instalación y una vez que llevó la instalación "
            "afectada a una condición segura, el Operador SCC o Jefe de Turno informa vía telefónica al Operador COC, "
            "para que éste a su vez informe al CDC-Sur, de acuerdo a lo establecido en PRO115.\n\n"
            "Excepto para los eventos en que la instalación afectada sea un paño de las Subestaciones Mulchén o Puente Negro, "
            "ya que estas instalaciones son operadas desde el COC y será el Operador del COC quien informe directamente "
            "al Coordinador Eléctrico Nacional."
        ),
        "inputs": [
            {"key": "hora_evento", **INPUTS_OPCIONALES["hora_evento"]},
        ],
        "checklist": [
            "Informar vía telefónica al Operador COC la desconexión y situación de la instalación.",
            "Gestionar/realizar el aviso al CDC-Sur según corresponda (vía COC o directo en casos Mulchén/Puente Negro).",
        ],
        "validacion": "¿Se informó la desconexión (COC y/o Coordinador según corresponda)?",
        "next": "T2",
    },
    "T2": {
        "type": "task",
        "titulo": "2 Registrar aviso de falla",
        "rol": "Operador o Jefe de Turno de SCC u Operador del COC",
        "descripcion": (
            "a) Ingresa un \"aviso de falla\" en Sistema SIGO, el que envía automáticamente un correo dirigido al COC, "
            "con copia a Subgerente de Sistemas Eléctricos, Ingenieros de la Subgerencia de Sistemas Eléctricos y listas "
            "de distribución definidas por cada Central y por la Gerencia de Transmisión.\n\n"
            "En caso que SIGO no esté disponible, completa formulario y lo envía por correo a la misma lista.\n\n"
            "b) Se comunica telefónicamente con el Operador COC para informar que ha ingresado el Aviso de Falla.\n\n"
            "Excepto para Mulchén o Puente Negro, donde el ingreso en SIGO lo realiza el Operador del COC.\n\n"
            "NOTA: La fecha y hora de inicio del aviso de falla queda establecida por el momento en que ocurre la falla.\n"
            "Plazo: A más tardar 1,45 horas desde ocurrido el evento de desconexión."
        ),
        "inputs": [
            {"key": "numero_aviso_sigo", **INPUTS_OPCIONALES["numero_aviso_sigo"]},
        ],
        "checklist": [
            "Ingresar un aviso de falla en SIGO (o completar formulario si SIGO no está disponible) y enviarlo a la distribución definida.",
            "Informar vía telefónica al Operador COC que se ingresó el Aviso de Falla.",
            "Si aplica: En casos Mulchén/Puente Negro, ingreso en SIGO lo realiza Operador COC.",
        ],
        "validacion": "¿Se registró el aviso de falla (SIGO/formulario) y se informó al COC?",
        "next": "T3",
    },
    "T3": {
        "type": "task",
        "titulo": "3 Registrar aviso en el Coordinador",
        "rol": "Operador COC",
        "descripcion": (
            "a) Revisa en SIGO (o formulario) que la información sea coherente y describa adecuadamente la falla. "
            "Si está incompleto, se comunica con Operador/Jefe de Turno para complementar.\n\n"
            "b) Registra aviso de falla en Sistema Neomante (web del Coordinador). El sistema crea un número correlativo.\n\n"
            "c) Informa al Despachador CDC el ingreso y el número correlativo; luego recibe instrucciones.\n\n"
            "d) Informa al Operador/Jefe de Turno el ingreso en el Coordinador y el número con el cual quedó registrado."
        ),
        "inputs": [
            {"key": "numero_neomante", **INPUTS_OPCIONALES["numero_neomante"]},
        ],
        "checklist": [
            "Revisar coherencia/completitud del aviso en SIGO (o formulario) y gestionar complemento si falta información.",
            "Registrar aviso de falla en Neomante y obtener número correlativo.",
            "Informar al Despachador CDC el ingreso y el número correlativo obtenido; recibir instrucciones.",
            "Informar al Operador/Jefe de Turno el ingreso en el Coordinador y número correlativo.",
        ],
        "validacion": "¿Se registró el aviso en Neomante y se informó a CDC y a la instalación?",
        "next": "T4",
    },
    "T4": {
        "type": "task",
        "titulo": "4 Verificar recepción de correo",
        "rol": "Operador o Jefe de Turno de SCC u Operador del COC",
        "descripcion": (
            "Verifica que el correo automático de notificación del sistema llega a los destinatarios, revisando el buzón "
            "de entrada en Outlook.\n\n"
            "Si no se recibe, elabora correo manual listando los parámetros registrados en SIGO y lo envía al Operador COC "
            "y a la Subgerencia de Sistemas Eléctricos."
        ),
        "checklist": [
            "Verificar recepción del correo automático en Outlook por los destinatarios.",
            "Si aplica: Elaborar y enviar correo manual con parámetros del aviso a Operador COC y Subgerencia de Sistemas Eléctricos.",
        ],
        "validacion": "¿Se verificó (o gestionó) la recepción de correo de notificación?",
        "next": "T5",
    },
    "T5": {
        "type": "task",
        "titulo": "5 Solicitar elaboración de Informe Antecedentes de Falla y envío de registros de protecciones",
        "rol": "Ingeniero de la Subgerencia de Sistemas Eléctricos",
        "descripcion": (
            "Solicita al Jefe de Operación de la Central o Complejo y/o al Jefe de Operación de Transmisión la elaboración "
            "del Informe de Antecedentes de Falla y el envío de registros de protecciones, vía correo con copia según corresponda, "
            "adjuntando el Formato de Informe Antecedentes de Falla.\n\n"
            "Plazo: El primer día hábil después de ocurrida la falla."
        ),
        "checklist": [
            "Solicitar por correo la elaboración del Informe de Antecedentes de Falla a los responsables correspondientes.",
            "Solicitar por correo el envío de registros de protecciones.",
            "Adjuntar Formato de Informe Antecedentes de Falla y copiar a los destinatarios definidos.",
        ],
        "validacion": "¿Se solicitó el informe de antecedentes y el envío de registros de protecciones?",
        "next": "T6",
    },
    "T6": {
        "type": "task",
        "titulo": "6 Solicitar extracción de registros",
        "rol": "Jefe de Operaciones / Supervisor o Ingeniero de Operaciones (Generación o Transmisión)",
        "descripcion": (
            "Solicita al personal de mantenimiento de Generación o Transmisión extraer ajustes, registros oscilográficos "
            "y de eventos de los relés de protección que hayan operado."
        ),
        "checklist": [
            "Solicitar al personal de mantenimiento extraer ajustes y registros (oscilográficos y de eventos) de relés que operaron.",
        ],
        "validacion": "¿Se solicitó la extracción de registros al personal de mantenimiento?",
        "next": "T7",
    },
    "T7": {
        "type": "task",
        "titulo": "7 Extraer registros",
        "rol": "Personal de mantenimiento de Generación o Transmisión",
        "descripcion": (
            "a) Extrae ajustes, registros oscilográficos y de eventos de relés que hayan operado en la falla, en formato original "
            "(por ejemplo COMTRADE, *.BEN, *.DEX, y/o *.pdf).\n\n"
            "b) Envía archivos por correo al Jefe/Supervisor/Ingeniero de Operaciones correspondiente."
        ),
        "checklist": [
            "Extraer ajustes y registros (oscilográficos y de eventos) de relés en formato original (COMTRADE/*.BEN/*.DEX/*.pdf, etc.).",
            "Enviar los archivos por correo a Jefe/Supervisor/Ingeniero de Operaciones correspondiente.",
        ],
        "validacion": "¿Se extrajeron y enviaron los registros al responsable de operaciones?",
        "next": "T8",
    },
    "T8": {
        "type": "task",
        "titulo": "8 Enviar registros de falla",
        "rol": "Jefe de Operaciones / Supervisor o Ingeniero de Operaciones (instalación afectada)",
        "descripcion": (
            "Envía los archivos con los registros de protecciones eléctricas por correo al Subgerente de Sistemas Eléctricos, "
            "con copia a los Ingenieros de la Subgerencia de Sistemas Eléctricos.\n\n"
            "Plazo: A más tardar un día hábil después de ocurrida la falla."
        ),
        "checklist": [
            "Enviar por correo los registros de protecciones al Subgerente de Sistemas Eléctricos, con copia a los ingenieros correspondientes.",
        ],
        "validacion": "¿Se enviaron los registros de falla a Sistemas Eléctricos?",
        "next": "T9",
    },
    "T9": {
        "type": "task",
        "titulo": "9 Analizar registros de falla",
        "rol": "Ingeniero de la Subgerencia de Sistemas Eléctricos",
        "descripcion": (
            "Realiza análisis de actuación de dispositivos de protección y control, y análisis razonado de la secuencia de eventos "
            "con base en la información recibida y otros antecedentes recabados."
        ),
        "checklist": [
            "Analizar la actuación de los dispositivos de protección y control.",
            "Analizar la secuencia de eventos en base a información recibida y antecedentes disponibles.",
        ],
        "validacion": "¿Se analizaron los registros y la secuencia de eventos?",
        "next": "T10",
    },
    "T10": {
        "type": "task",
        "titulo": "10 Elaborar Informe de Antecedentes de Falla",
        "rol": "Jefe de Operaciones / Supervisor o Ingeniero de Operaciones (instalación afectada)",
        "descripcion": (
            "a) Elabora Informe de Antecedentes de Falla si la Subgerencia de Sistemas Eléctricos lo solicita, de acuerdo al Formato "
            "recibido y contenido descrito en Anexo 8.1. Puede incorporar más información si es necesario.\n\n"
            "b) Envía el informe por correo al Subgerente de Sistemas Eléctricos con copia a los ingenieros correspondientes.\n\n"
            "Plazo: A más tardar dos días hábiles después de ocurrida la falla."
        ),
        "checklist": [
            "Elaborar Informe de Antecedentes de Falla conforme al Formato recibido (Anexo 8.1) si fue solicitado.",
            "Enviar el informe por correo al Subgerente de Sistemas Eléctricos con copia a los ingenieros correspondientes.",
        ],
        "validacion": "¿Se elaboró y envió el Informe de Antecedentes de Falla (si fue solicitado)?",
        "next": "T11",
    },
    "T11": {
        "type": "task",
        "titulo": "11 Elaborar Informe de Falla",
        "rol": "Ingeniero de la Subgerencia de Sistemas Eléctricos",
        "descripcion": (
            "Elabora Informe de Falla de acuerdo a lo establecido en Anexo 8.2 y lo almacena en el Gestor Documental: "
            "División Generación \\ 02_Gerencia de Sistemas Eléctricos \\ 24_Informes de falla.\n\n"
            "Aplica para eventos donde hubo pérdida de consumo o propagación a terceros (o desde terceros a Colbún)."
        ),
        "checklist": [
            "Elaborar Informe de Falla según Anexo 8.2.",
            "Almacenar el Informe de Falla en el Gestor Documental (División Generación \\ 02_Gerencia de Sistemas Eléctricos \\ 24_Informes de falla).",
            "Si aplica: Considerar casos con pérdida de consumo o propagación a terceros.",
        ],
        "validacion": "¿Se elaboró y almacenó el Informe de Falla según Norma Técnica?",
        "next": "T12",
    },
    "T12": {
    "type": "task",
    "titulo": "12 - Enviar informe de falla a encargados de Colbún",
    "rol": "Subgerente de Sistemas Eléctricos",
    "descripcion": (
        "Envía Informe de Falla (archivo *.doc) por correo electrónico a los encargados "
        "de Colbún S.A. y Colbún Transmisión S.A. ante el Coordinador "
        "(Titular y Suplente), dependiendo de la instalación afectada.\n\n"
        "Plazo: A más tardar a medio día del quinto día hábil después de ocurrida la falla."
    ),
    "checklist": [
        "Enviar Informe de Falla (*.doc) a encargados Colbún ante el Coordinador (Titular y Suplente).",
        "Verificar cumplimiento del plazo (máximo medio día del quinto día hábil)."
    ],
    "validacion": "¿Se envió el Informe de Falla a los encargados dentro del plazo establecido?",
    "next": "T13",
},

"T13": {
    "type": "task",
    "titulo": "13 - Revisar Informe de Falla",
    "rol": "Encargados ante el Coordinador (Titular y Suplente)",
    "descripcion": (
        "a) Revisa Informe de Falla preparado por la Subgerencia de Sistemas Eléctricos. "
        "En caso de dudas u observaciones, las resuelve en conjunto con el Subgerente "
        "hasta acordar una versión final.\n\n"
        "b) Gestiona el envío del Informe de Falla al Coordinador, adjuntándolo "
        "al aviso de falla a través del sistema Neomante."
    ),
    "checklist": [
        "Revisar Informe de Falla preparado por la Subgerencia de Sistemas Eléctricos.",
        "Resolver observaciones en conjunto con Subgerente de Sistemas Eléctricos (si aplica).",
        "Gestionar envío del Informe de Falla al Coordinador a través de Neomante."
    ],
    "validacion": "¿Se revisó y gestionó el envío del Informe de Falla al Coordinador?",
    "next": "END_OK",
},

    "END_OK": {
        "type": "end",
        "titulo": "🏁 Proceso finalizado",
        "rol": "HMI",
        "descripcion": "Se completaron los pasos del PRO114 (2.1.1 a 2.1.11). Puede exportar el JSON auditable si lo requiere.",
        "mensaje": "Fin del procedimiento.",
        "estado_final": FINALIZADO,
        "show_tables": True,
    },
}

# ------------------------------------------------------------
# MOTOR HMI (mantener formato / UI estándar)
# ------------------------------------------------------------

class PRO114HMI:
    def __init__(self):
        self.nodo_id = "T1"
        self.estado = EN_CURSO
        self.run_id = str(uuid.uuid4())
        self.start_ts = _now_iso()
        self.end_ts = None

        self.inputs = {}
        self.decisiones = []
        self.logs = []
        self.historial = []

        self.output = widgets.Output()

        # Botones estándar (SIN STOP)
        self.btn_si = widgets.Button(description="SÍ", button_style="success", layout={"width":"49%","height":"44px"})
        self.btn_no = widgets.Button(description="NO", button_style="danger", layout={"width":"49%","height":"44px"})
        self.btn_volver = widgets.Button(description="⬅ Volver al paso anterior", layout={"width":"100%","height":"40px"})
        self.btn_exportar = widgets.Button(description="Exportar JSON (trazabilidad)", icon="download", layout={"width":"100%","height":"40px"})

        self.msg_box = widgets.HTML("")
        self._check_widgets = []
        self._decision_widget = None
        self._input_widgets = []

        # Wire
        self.btn_si.on_click(self._on_si)
        self.btn_no.on_click(self._on_no)
        self.btn_volver.on_click(self._on_volver)
        self.btn_exportar.on_click(self._on_exportar)

        display(self.output)
        self.iniciar()

    def _log(self, tipo, data=None):
        self.logs.append({
            "ts": _now_iso(),
            "tipo": tipo,
            "nodo": self.nodo_id,
            "data": data or {}
        })

    def _push_history(self):
        self.historial.append(self.nodo_id)

    def _pop_history(self):
        if self.historial:
            return self.historial.pop()
        return None

    def _clear_msg(self):
        self.msg_box.value = ""

    def _msg(self, text, kind="warn"):
        if kind == "ok":
            self.msg_box.value = f"<div style='margin-top:10px;padding:10px;border-radius:10px;background:#dcfce7;border:1px solid #22c55e;color:#14532d;font-size:12px;'><b>{text}</b></div>"
        else:
            self.msg_box.value = f"<div style='margin-top:10px;padding:10px;border-radius:10px;background:#fee2e2;border:1px solid #ef4444;color:#7f1d1d;font-size:12px;'><b>{text}</b></div>"

    def _render_header(self, n):
        badge = f"<span style='display:inline-block;padding:4px 10px;border-radius:999px;background:#eef2ff;border:1px solid #c7d2fe;font-size:12px;color:black;'><b>ROL:</b> {n.get('rol','')}</span>"
        return widgets.HTML(f"""
        <div style="padding:14px;border-radius:12px;background:#f8fafc;border:1px solid #e2e8f0;">
            <div style="font-size:12px;color:#0f172a;"><b>PRO114</b> – Elaboración de Informes de Falla según Norma Técnica</div>
            <div style="margin-top:6px;font-size:20px;color:#0f172a;"><b>{n.get('titulo','')}</b></div>
            <div style="margin-top:8px;">{badge}</div>
            <div style="margin-top:10px;color:#0f172a;font-size:13px;line-height:1.35;white-space:pre-wrap;">{n.get('descripcion','')}​</div>
        </div>
        """)

    def _render_inputs(self, n):
        self._input_widgets = []
        if not n.get("inputs"):
            return widgets.VBox([])
        items = []
        items.append(widgets.HTML("""
        <div style="margin-top:12px;padding:12px;border-radius:12px;border:1px solid #e2e8f0;background:#ffffff;">
            <div style="font-size:13px;color:#0f172a;"><b>🧩 CAMPOS (opcionales)</b></div>
            <div style="margin-top:6px;font-size:12px;color:#0f172a;opacity:0.85;">
                Estos campos se guardan en el JSON exportable. Si los dejas vacíos, no bloquean.
            </div>
        </div>
        """))
        for spec in n["inputs"]:
            key = spec["key"]
            label = spec.get("label", key)
            w = widgets.Text(
                description=label,
                layout=widgets.Layout(width="100%"),
                style={"description_width":"initial"}
            )
            if key in self.inputs:
                w.value = self.inputs.get(key,"")
            self._input_widgets.append((spec, w))
            items.append(w)
        return widgets.VBox(items)

    def _render_task(self, n):
        valid = n.get("validacion","")

        self._check_widgets = [
            widgets.Checkbox(description=item, value=False, layout=widgets.Layout(width="100%"))
            for item in (n.get("checklist",[]) or [])
        ]

        checklist_box = widgets.VBox([])
        if self._check_widgets:
            checklist_box = widgets.VBox([
                widgets.HTML("""
                <div style="margin-top:12px;padding:12px;border-radius:12px;border:1px solid #e2e8f0;background:#ffffff;">
                    <div style="font-size:13px;color:#0f172a;"><b>🧾 ACCIONES (Checklist)</b></div>
                    <div style="margin-top:6px;font-size:12px;color:#0f172a;opacity:0.9;">
                        Ítems marcados como <b>“Si aplica”</b> no son obligatorios para avanzar.
                    </div>
                </div>
                """),
                widgets.VBox(self._check_widgets)
            ])

        valid_box = widgets.HTML(f"""
            <div style="margin-top:12px;padding:12px;border-radius:12px;border:2px solid #0ea5e9;background:#ffffff;">
                <div style="font-size:13px;color:#0f172a;"><b>✅ ¡VALIDACIÓN!</b></div>
                <div style="margin-top:8px;font-size:15px;color:#0f172a;"><b>{valid}</b></div>
                <div style="margin-top:6px;font-size:12px;color:#0f172a;">Confirma con <b>SÍ</b> para avanzar. Si respondes <b>NO</b>, el paso queda bloqueado.</div>
            </div>
        """)

        inputs_box = self._render_inputs(n)
        return widgets.VBox([inputs_box, checklist_box, valid_box])

    def _checklist_obligatorio_ok(self):
        oblig = []
        for cb in self._check_widgets:
            if not _es_opcional_por_si_aplica(cb.description):
                oblig.append(cb)
        return all(cb.value for cb in oblig) if oblig else True

    def _collect_inputs(self, n):
        if not n.get("inputs"):
            return True, ""
        for spec, w in self._input_widgets:
            key = spec["key"]
            required = bool(spec.get("required", False))
            val = (w.value or "").strip()
            if required and not val:
                return False, f"Debe completar: {spec.get('label', key)}"
            # Guardar solo si hay valor (para compactar inputs)
            if val:
                self.inputs[key] = val
        return True, ""

    def _render_tables_if_end(self, n):
        if n.get("type") != "end" or not n.get("show_tables", False):
            return widgets.VBox([])

        # Compact tables
        inputs_rows = "".join([f"<tr><td style='padding:4px 6px;'><b>{k}</b></td><td style='padding:4px 6px;'>{(v or '')}</td></tr>" for k, v in self.inputs.items()])
        if not inputs_rows:
            inputs_rows = "<tr><td colspan='2' style='padding:6px;'>(sin inputs)</td></tr>"

        dec_rows = "".join([f"<tr><td style='padding:4px 6px;'>{d.get('ts','')}</td><td style='padding:4px 6px;'>{d.get('nodo','')}</td><td style='padding:4px 6px;'>{d.get('seleccion','')}</td></tr>" for d in self.decisiones])
        if not dec_rows:
            dec_rows = "<tr><td colspan='3' style='padding:6px;'>(sin decisiones)</td></tr>"

        return widgets.VBox([
            widgets.HTML(f"""
            <div style="margin-top:12px;padding:12px;border-radius:12px;border:1px solid #e2e8f0;background:#ffffff;">
                <div style="font-size:13px;color:#0f172a;"><b>📌 INPUTS</b></div>
                <table style="width:100%;border-collapse:collapse;margin-top:8px;font-size:12px;">
                    <thead><tr>
                        <th style="border:1px solid #e2e8f0;padding:4px 6px;text-align:left;">Campo</th>
                        <th style="border:1px solid #e2e8f0;padding:4px 6px;text-align:left;">Valor</th>
                    </tr></thead>
                    <tbody>{inputs_rows}</tbody>
                </table>
            </div>
            """),
            widgets.HTML(f"""
            <div style="margin-top:12px;padding:12px;border-radius:12px;border:1px solid #e2e8f0;background:#ffffff;">
                <div style="font-size:13px;color:#0f172a;"><b>🧭 DECISIONES / EVENTOS</b></div>
                <table style="width:100%;border-collapse:collapse;margin-top:8px;font-size:12px;">
                    <thead><tr>
                        <th style="border:1px solid #e2e8f0;padding:4px 6px;text-align:left;">Timestamp</th>
                        <th style="border:1px solid #e2e8f0;padding:4px 6px;text-align:left;">Nodo</th>
                        <th style="border:1px solid #e2e8f0;padding:4px 6px;text-align:left;">Selección</th>
                    </tr></thead>
                    <tbody>{dec_rows}</tbody>
                </table>
            </div>
            """),
        ])

    def _render_footer(self):
        return widgets.VBox([
            widgets.HBox(
                [self.btn_si, self.btn_no],
                layout=widgets.Layout(justify_content="space-between", gap="8px", margin="10px 0")
            ),
            self.btn_volver,
            widgets.HTML("<div style='height:8px;'></div>"),
            self.btn_exportar,
            widgets.HTML("<div style='height:8px;'></div>"),
            self.msg_box,
        ])

    def _render(self):
        with self.output:
            self.output.clear_output()
            self._clear_msg()

            n = NODOS[self.nodo_id]
            header = self._render_header(n)

            if n["type"] == "task":
                body = self._render_task(n)
                self._decision_widget = None
            else:
                body = widgets.HTML(f"""
                <div style="margin-top:12px;padding:12px;border-radius:12px;border:1px solid #e2e8f0;background:#ffffff;">
                    <div style="font-size:14px;color:#0f172a;"><b>{n.get('mensaje','')}</b></div>
                </div>
                """)
                self._decision_widget = None
                self._check_widgets = []
                self._input_widgets = []

            end_tables = self._render_tables_if_end(n)
            footer = self._render_footer()

            # ✅ Colab-friendly: single root container
            display(widgets.VBox([header, body, end_tables, footer]))

    def iniciar(self):
        self._render()

    # ---------- eventos ----------
    def _on_si(self, _):
        n = NODOS[self.nodo_id]
        if n["type"] == "end":
            self._msg("Este es un nodo final.", "ok")
            return

        # Validar checklist + inputs
        if not self._checklist_obligatorio_ok():
            self._msg("Debe completar las acciones obligatorias antes de avanzar.", "warn")
            self._log("VALIDACION_FALLA", {"mensaje": "checklist obligatorio incompleto"})
            return

        ok_inputs, msg = self._collect_inputs(n)
        if not ok_inputs:
            self._msg(msg, "warn")
            self._log("VALIDACION_FALLA", {"mensaje": msg})
            return

        self.decisiones.append({"ts": _now_iso(), "nodo": self.nodo_id, "seleccion": "OK"})
        self._push_history()
        nxt = n.get("next")
        self._log("AVANZA", {"next": nxt})
        self.nodo_id = nxt

        if NODOS[self.nodo_id]["type"] == "end":
            self.estado = NODOS[self.nodo_id].get("estado_final", FINALIZADO)
            self.end_ts = _now_iso()

        self._render()

    def _on_no(self, _):
        n = NODOS[self.nodo_id]
        if n["type"] == "end":
            self._msg("Este es un nodo final.", "ok")
            return
        self.estado = BLOQUEADO
        self._log("BLOQUEA", {"motivo": "Respuesta NO en validación"})
        self._msg("Paso bloqueado: respondió NO en la validación.", "warn")
        self._render()

    def _on_volver(self, _):
        prev = self._pop_history()
        if prev is None:
            self._msg("No hay paso anterior.", "warn")
            return
        self.nodo_id = prev
        self.estado = EN_CURSO
        self._log("VOLVER", {"to": prev})
        self._render()

    def _on_exportar(self, _):
        payload = {
            "proceso": "PRO114 – Elaboración de Informes de Falla según Norma Técnica",
            "run_id": self.run_id,
            "estado": self.estado,
            "start_ts": self.start_ts,
            "end_ts": self.end_ts,
            "current_node": self.nodo_id,
            "history_stack": list(self.historial),
            "decisiones": list(self.decisiones),
            "inputs": dict(self.inputs),
            "logs": list(self.logs),
            "export_ts": _now_iso(),
        }
        pretty = json.dumps(payload, ensure_ascii=False, indent=2)
        self.msg_box.value = f"""
        <div style='margin-top:10px;padding:10px;border-radius:10px;background:#dcfce7;border:1px solid #22c55e;color:#14532d;font-size:12px;'>
            <b>📦 Export JSON (trazabilidad)</b>
            <pre style='white-space:pre-wrap;margin-top:10px;color:#14532d;font-size:12px;'>{pretty}</pre>
        </div>
        """

# Ejecutar
hmi = PRO114HMI()

Output()